In [ ]:
import numpy as np
from mat4py import loadmat
import scipy.io
from copy import deepcopy
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.model_selection import StratifiedKFold

import tensorflow 
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

import matplotlib.pyplot as plot
from sklearn.metrics import confusion_matrix
import csv
from tabnanny import verbose